<a href="https://www.kaggle.com/code/mikedelong/q-and-a-with-mistral-7b-no-accelerator?scriptVersionId=159641931" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

I have been wanting to play with Mistral-7B but I don't want to up my precious accelerator credits. Here's a simple demo that runs in a standard CPU environment with no accelerators. This is simplified from the demo https://developer-service.blog/building-a-chat-application-with-chainlit-and-mistral-7b-on-cpu/ 

It is slow but useful as a proof of concept.

In [1]:
!pip install ctransformers

  Obtaining dependency information for ctransformers from https://files.pythonhosted.org/packages/14/50/0b608e2abee4fc695b4e7ff5f569f5d32faf84a49e322034716fa157d1cf/ctransformers-0.2.27-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.1 MB/s eta 0:00:00


In [2]:
# setting up the model takes less than thirty seconds
from arrow import now
from ctransformers import AutoModelForCausalLM

MAX_NEW_TOKENS = 2048
MODEL_PATH = 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
MODEL_FILE = 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'
MODEL_TYPE = 'mistral'
TEMPERATURE = 0.7
THREADS = 4

setup_start = now()
model = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id=MODEL_PATH, model_file=MODEL_FILE, model_type=MODEL_TYPE,
                                             temperature=0.7, gpu_layers=0, stream=False, threads=THREADS, max_new_tokens=MAX_NEW_TOKENS)
print('built model in {}'.format(now() - setup_start))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

built model in 0:00:19.241763


In [3]:
# our output comes back as a stream of tokens; we want to format it as a list of sentences
from arrow import now
from nltk.tokenize import sent_tokenize

def respond(question: str):
    respond_start = now()
    print('Q: {}'.format(question))
    result = ''.join(model(prompt=question,)).replace('\n', ' ')
    sentences = sent_tokenize(text=result, language='english')
    sentences = [sentence for sentence in sentences if len(sentence.strip()) > 0]
    for answer in sentences:
        print('{}'.format(answer.strip()))
    print('done in {}'.format(now() - respond_start))

print(now())

2024-01-19T17:05:48.959424+00:00


In [4]:
# let's start with an easy one:
respond(question='What is two plus two?')

Q: What is two plus two?
Answer: 4
done in 0:00:09.682492


In [5]:
# now let's try a trick question:
respond(question='Do these jeans make me look fat?')

Q: Do these jeans make me look fat?
A: No, these jeans do not make you look fat.
done in 0:00:14.238224


In [6]:
# let's ask for a recipe
respond(question='How do I make a dry gin martini?')
# the key is the ratio of gin to vermouth; it should be 5:1
# Mistral gets the ingredients right but is fuzzy on the quantities
# and make take up to a minute to respond

Q: How do I make a dry gin martini?
1.
Fill a mixing glass with ice.
2.
Pour in 1 to 1.5 oz of dry gin (such as Hendrick's or Botanist).
3.
Add 0.75 to 1 oz of dry vermouth (such as Dolin or Carpano Antica).
4.
Stir the mixture well for about 30 seconds until chilled.
5.
Strain the mixture into a chilled martini glass.
6.
Garnish with a lemon twist or an olive, if desired.
Note: The ratio of gin to vermouth is a matter of personal preference.
Some people prefer a more gin-heavy martini, while others prefer a slightly sweeter vermouth-based drink.
Feel free to adjust the ratio according to your taste.
done in 0:01:22.472541


In [7]:
# let's ask an exact question about environmental regulations
respond(question='What are Scope 3 emissions?')
# sometimes Mistral tells us what all three scopes are; sometimes not.

Q: What are Scope 3 emissions?
Answer: Scope 3 emissions refer to all greenhouse gas emissions that are not directly attributed to a company's owned or controlled operations, but rather to its value chain activities, including those of its suppliers and customers.
These emissions can include transportation-related activities, energy consumption by suppliers and customers, waste disposal, and product use and end-of-life disposal.
Companies often struggle to accurately measure and reduce these emissions, as they are spread across a wide range of activities and stakeholders.
done in 0:00:44.235670


In [8]:
# this one seems like it should be easy but Mistral is hit or miss
respond(question='What were the last five states admitted to the Union and when were they each admitted?')

Q: What were the last five states admitted to the Union and when were they each admitted?
The last five states to be admitted to the United States were: 1.
Alaska - July 3, 1958 2.
Hawaii - August 21, 1959 3.
Arizona - February 14, 1912 4.
New Mexico - September 6, 1912 5.
California - September 9, 1850
done in 0:00:42.629807


In [9]:
# this is a matter of opinion, but the term means a particular thing and Mistral usually gets it right
respond(question='In Rome who were the Five Good Emperors?')

Q: In Rome who were the Five Good Emperors?
1) Nerva 2) Trajan 3) Hadrian 4) Antoninus Pius 5) Marcus Aurelius  Answer: 1) Nerva, 2) Trajan, 3) Hadrian, 4) Antoninus Pius, and 5) Marcus Aurelius.
done in 0:00:33.465829


In [10]:
# this is a matter of opinion and Mistral usually answers it well
respond(question='Who was the Fifth Beatle?')

Q: Who was the Fifth Beatle?
The Fifth Beatle refers to various individuals who were considered an integral part of The Beatles, but did not officially become a band member.
These include George Martin (the producer), Klaus Voormann (a German graphic artist and bassist who worked with them on album covers and other projects), and Harrison Brown (who was credited as the "Fifth Beatle" for his work as the assistant film director on the 1967 film Help!
).
done in 0:00:41.684817


In [11]:
# Mistral can take a while (minutes?) to answer this but the answers are generally good.
respond(question='What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?')

Q: What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?
I think I’ll start with some basic pieces:  - Jeans (blue, black, gray) - T-shirts (white, gray, black) - White button down shirts - Sweaters (gray, navy, black) - Leather dress shoes - A good quality belt - A good quality watch  What else should I include?
Should I get a suit?
I’m 25 and just graduated from college.
I plan on finding my first job pretty quickly.
What else should I consider buying?
Any specific pieces of clothing or accessories that are must haves for me?
Thanks!
done in 0:01:04.049478


In [12]:
# Mistral sometimes misunderstands this question and rephrases it as a question
respond(question='I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?')

Q: I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?
I'm not sure what you mean by "build a minimal wardrobe," but I assume that you want to create a capsule wardrobe.
To start, consider the following items:  1.
T-shirts and tanks - look for ones made of high-quality materials like cotton or linen, which will last longer and feel more comfortable.
2.
Jeans - good jeans can be dressed up or down depending on the occasion, and they'll become a wardrobe staple.
3.
Dresses - a few simple dresses in different colors and styles can take you from casual to formal.
4.
Blazers and jackets - these will add versatility to your wardrobe, allowing you to layer up for cooler weather or transition an outfit from day to night.
5.
Skirts and pants - depending on your personal style, consider adding a few skirts or pants to your wardrobe.
6.
Shoes - invest in good quality shoes that will last for years, such as classic sneakers, fl

In [13]:
# Some SOTA engines get this right but this still seems to cause problems
# a correct answer is American Football or Rugby or Australian Rules Football;
# hockey, badminton, and ultimate frisbee are popular but incorrect answers
respond(question='What is a sport that is played with a ball that is not round?')

Q: What is a sport that is played with a ball that is not round?
A. Rugby B.
Tennis C. Basketball D. Golf Answer: C. Basketball
done in 0:00:17.167376


In [14]:
# No. Just no.
respond(question='What are five kinds of balls that are not round?')

Q: What are five kinds of balls that are not round?
1.
A cube is an example of a ball that is not round.
Its shape has six square faces meeting at right angles.
2.
A soccer ball is another kind of non-round ball, designed to be spherical but with indented panels for better grip and control during play.
3.
An egg is a type of spheroid ball that's not perfectly round due to the air pocket inside it which gives it an oval shape.
4.
A tennis ball has a fuzzy covering on its surface making it slightly irregular in shape, although generally spherical.
5.
A golf ball has dimples on its surface which reduce air resistance and help increase distance when hit, but also make it deviate from a perfect sphere.
done in 0:01:07.305444


In [15]:
# If we just ask for one essential ingredient Mistral will say espresso
respond(question='What are the essential ingredients of a cafe latte?')

Q: What are the essential ingredients of a cafe latte?
Essential ingredients for a café latte include brewed coffee, steamed milk, and foam.
Some variations may also include flavors such as syrups or whipped cream.
done in 0:00:18.603423


In [16]:
# the answers to this are sometimes bizarre
respond(question='What fruit does a lime taste most like?')

Q: What fruit does a lime taste most like?
A lime tastes most like itself.
It has a tart, tangy flavor with a hint of sweetness that is distinct from other citrus fruits like lemons or oranges.
done in 0:00:19.080874


In [17]:
# this one is hit or miss; answers can vary by several orders of magnitude
respond(question='How much do fingernails grow in a year?')

Q: How much do fingernails grow in a year?
Fingernails typically grow about 0.5 mm (2/6 of an inch) per month.
To calculate how much they grow in a year, we need to multiply this monthly rate by the number of months in a year, which is 12:  Growth in a year = Growth per month * Number of months                          = 0.5 mm/month * 12 months                          = 6 mm  However, it's important to note that this is just an average estimate and individual fingernails may grow at different rates due to factors like genetics, overall health, and nail care habits.
done in 0:01:10.182537


In [18]:
# this is a trick question and Mistral answers quickly and never gets it right
respond(question='When did Hilary Clinton serve as President of the United States?')

Q: When did Hilary Clinton serve as President of the United States?
Answer: 2009-2017
done in 0:00:09.931110


In [19]:
# I saw this one on the Web; even SOTA engines give bizarre answers to this one
respond(question='What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?')

Q: What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?
1.
She Loves You (mentions "she loves you" and "means it") 2.
Mr. Jolly (mentions a "jolly old fellow") 3.
Blackbird (mentions a "blackbird singing in the trees") 4.
Lucy in the Sky with Diamonds (mentions a "little yellow dog") 5.
Penny Lane (mentions a "piggy in the sky") 6.
Ticket to Ride (mentions a "motorcar" and a "carrick") 7.
All My Loving (mentions a "poodle" and a "dog" being unfaithful)
done in 0:01:05.494333


In [20]:
# There doesn't seem to be a way to phrase this question so Mistral consistently gets it right
respond(question='Can you name a country where the name of the country starts with the letter A but ends with a different letter?')

Q: Can you name a country where the name of the country starts with the letter A but ends with a different letter?
Answer: Bangladesh (name starts with A, ends with D)
done in 0:00:15.297768


In [21]:
# Mistral usually does pretty well answering this one
respond(question='What do people mean when they say money is not real?')

Q: What do people mean when they say money is not real?
When people say that money is not real, they are expressing the idea that money is merely a human construct and has no inherent value.
They may argue that money is based on faith in the banking system and government currency, and therefore it can be easily manipulated and devalued.
Some may also contend that money distracts from real values such as love, compassion, and personal growth.
However, despite these arguments, money remains an essential part of many societies and economies, serving as a medium of exchange, unit of account, and store of value.
done in 0:00:50.537006


In [22]:
# But not this one
respond(question='What was the Mother of All Demos and where did it occur?')

Q: What was the Mother of All Demos and where did it occur?
The Mother of All Demos was a demonstration held at the 1964 New York World's Fair.
It showcased IBM's Model/3 computer, which had been developed specifically for business use.
The demonstration was considered revolutionary as it demonstrated the potential of computers for everyday use in businesses and homes.
done in 0:00:32.440692


In [23]:
# Is Mistral a Luddite?
respond(question='If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?')

Q: If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?
A: Less of it
done in 0:00:11.807694


In [24]:
# if we're not careful Mistral will tell us GDP is a measure of government spending
respond(question='What does GDP measure most accurately?')

Q: What does GDP measure most accurately?
A.
The total value of goods and services produced B.
The total income earned by households and businesses C. The total cost of goods and services consumed D. The total amount of capital accumulated over time  Answer: A.
The total value of goods and services produced
done in 0:00:27.490392


In [25]:
# sometimes Mistral says one and sometimes the other
respond(question='Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?',)

Q: Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?
A planned economy is one in which the government controls all aspects of production, distribution, and consumption.
A free market is one in which prices are determined by supply and demand, and individuals and businesses make their own economic decisions.
There is no definitive answer to this question as it depends on a variety of factors such as the specific country, time period, and economic goals.
However, some studies suggest that a free market economy generally leads to better economic outcomes for the median person, including higher wages, greater job opportunities, and more choices in products and services.
In a free market economy, businesses are incentivized to produce goods and services efficiently and at lower costs, which can lead to lower prices for consumers.
As wages are determined by supply and demand in a free market, there may be greater press

In [26]:
# When we stand on our heads our heads are down not up
respond(question='If I do a handstand what part of my body is on top?')

Q: If I do a handstand what part of my body is on top?
My feet
done in 0:00:06.752387


In [27]:
# doing a handstand and standing on my head are different but similar
respond(question='If I am standing on my head what part of my body is on top?')

Q: If I am standing on my head what part of my body is on top?
My bottom.
done in 0:00:07.873480


In [28]:
# apparently without the 'standing on' clue this is a difficult question
respond(question='If I do a head stand what part of my body is on top?')

Q: If I do a head stand what part of my body is on top?
My neck, shoulder and head are all on top.
done in 0:00:09.060220


So what's the takeaway?
1. Mistral-7B seems like a miracle; the very idea that something this powerful is available for free download and fast enough to use in this environment seemed like science fiction five years ago.
2. Mistral-7B doesn't say "I don't know" in the way a person might and doesn't always respond well to trick questions or ambiguous questions. 
3. Because Mistral-7B will give answers that vary substantially to some fairly simple factual questions it is a good idea to ask the same question several times and QA the responses.
4. Mistral-7B is better at narrative answers than factual answers. 
5. Mistral-7B has unknown unknowns that should be treated with care.
6. Mistral-7B can do simple arithmetic embedded in a question (see our fingernails question above) but doesn't always do it correctly, and flounders on complex reasoning questions (e.g. the Beatles/animials question).
7. Mistral-7B has no real grasp of the physical world (the handstand/head stand questions).